# SQL in Jupyter

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Checking existing table in database

In [2]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Printing first 5 rows of facts table

In [3]:
%%sql
SELECT * FROM facts LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary statistics

In [61]:
%%sql

SELECT 
MIN(population), 
MAX(population),
MIN(population_growth), 
MAX(population_growth)
FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Exploring outliers

In [8]:
%%sql

SELECT *
FROM facts
WHERE population IN (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [9]:
%%sql

SELECT *
FROM facts
WHERE population IN (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Average population and area

In [12]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Exploring population area ratio

In [17]:
%%sql
SELECT name as Country, area_land as Area, population as Population
FROM facts
WHERE population > (SELECT AVG(population) FROM facts) AND 
area_land < (SELECT AVG(area_land) FROM facts)
ORDER BY Population DESC

 * sqlite:///factbook.db
Done.


Country,Area,Population
Bangladesh,130170,168957745
Japan,364485,126919659
Philippines,298170,100998376
Vietnam,310070,94348835
Germany,348672,80854408
Thailand,510890,67976405
United Kingdom,241930,64088222


## Top 5 the most dens countries.

In [38]:
%%sql
SELECT name as Country, ROUND(CAST(population as Float) / CAST(area_land as Float),2) as 'People_per_square_m'
FROM facts
ORDER BY People_per_square_m DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


Country,People_per_square_m
Macau,21168.96
Monaco,15267.5
Singapore,8259.78
Hong Kong,6655.27
Gaza Strip,5191.82


## Countries with the highest ration of water to land.

In [46]:
%%sql

SELECT name as Country, ROUND(CAST(area_water as Float) / CAST(area_land as Float),2) as area_ratio
FROM facts
ORDER BY area_ratio DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


Country,area_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


Top 2 are some teritories belonging to different countries like UK and US.
Among single countries Puerto Rico is having more then half its territory on sea.

In [50]:
%%sql
SELECT name as Country, birth_rate, death_rate
FROM facts
WHERE birth_rate < death_rate
ORDER BY birth_rate / death_rate

 * sqlite:///factbook.db
Done.


Country,birth_rate,death_rate
Bulgaria,8.92,14.44
Serbia,9.08,13.66
Latvia,10.0,14.31
Lithuania,10.1,14.27
Hungary,9.16,12.73
Monaco,6.65,9.24
Slovenia,8.42,11.37
Ukraine,10.72,14.46
Germany,8.47,11.42
Saint Pierre and Miquelon,7.42,9.72


Here we can see all countries in which death rate is higher then birth rate, sorted descending.

In [58]:
%%sql
SELECT name as Country, ROUND(population * population_growth/100,0) as people_growth
FROM facts
ORDER BY people_growth DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


Country,people_growth
World,78370092.0
India,15270686.0
China,6153684.0
Nigeria,4448270.0
Pakistan,2906653.0
Ethiopia,2874562.0
Bangladesh,2703324.0
United States,2506677.0
Indonesia,2355142.0
"Congo, Democratic Republic of the",1944691.0


Here we can see top 10 countries in which next year the most people will come.